In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [54]:
df_customer = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_price = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_meta = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_meta_cleaned_ver2.csv", engine= "python", encoding='utf-8')

In [46]:
df_download.head(5)

,customer_id,item_id,down_year,down_date,weekday,inf_year,down_price
0,C5001,I-1010,2014,02-21-2014,6,1.051872,0.946685
1,C5001,I-1014,2014,04-04-2014,6,1.051872,0.946685
2,C5001,I-1017,2014,01-19-2014,1,1.051872,0.946685
3,C5001,I-1028,2014,01-21-2014,3,1.051872,0.946685
4,C5001,I-1035,2014,03-11-2014,3,1.051872,0.899351


In [47]:
df_download['down_price'].describe()

count    73085.000000
mean         0.974824
std          0.089535
min          0.810000
25%          0.918900
50%          0.931777
75%          1.021000
max          1.262247
Name: down_price, dtype: float64

In [67]:
# 고객별 매출
rev_customer = df_download.groupby(['customer_id'])['down_price'].sum()
df_customer = pd.merge(df_customer,rev_customer, on='customer_id',how='left')
df_customer = df_customer.rename(columns={'down_price':'customer_sales'})
# df_customer.drop(['down_price_x','down_price_y'],inplace=True,axis=1)

In [68]:
# 영화별 매출
rev_movie = df_download.groupby(['item_id'])['down_price'].sum()
df_meta = pd.merge(df_meta,rev_movie, on='item_id',how='left')
df_meta = df_meta.rename(columns={'down_price':'movie_down_sales'})

In [69]:
df_meta.columns

Index(['movie_id', 'title', 'release_year', 'runtime', 'mpa_rating',
       'imdb_score', 'dvd_sales', 'blu_sales', 'total_sales', 'legs', 'share',
       'inf_income_usa', 'theater_opening', 'theater_total', 'series', 'src',
       'creative_type', 'director', 'actor', 'writer', 'description', 'synop',
       'contract_price', 'studio_score', 'price_class', 'awards', 'poster',
       'metascore', 'release_dvd', 'big_awards_num', 'awards_win_num',
       'awards_nomin_num', 'reviews_users', 'reviews_critics', 'prd_mthd',
       'prd_company', 'release_date', 'genre', 'country', 'language', 'budget',
       'series_new', 'kwrds', 'actor_1', 'actor_2', 'actor_3', 'actor_4',
       'income_opening', 'votes', 'income_usa', 'income_int', 'income_ww',
       'item_id', 'inv_exist', 'contract_year', 'inf', 'genre_action',
       'genre_adventure', 'genre_drama', 'genre_sci-fi', 'genre_fantasy',
       'genre_thriller', 'genre_animation', 'genre_family', 'genre_musical',
       'genre_comedy',

In [34]:
rev_customer

customer_id
C5001    10.997324
C5002    23.661645
C5003    48.821681
C5004    38.822686
C5005    36.841977
           ...    
C8273     9.990000
C8274    21.770000
C8275     4.410000
C8276    13.480000
C8277     3.800000
Name: down_price, Length: 2738, dtype: float64

In [35]:
df_customer['customer_id']

0       C5001
1       C5002
2       C5003
3       C5004
4       C5005
        ...  
3272    C8273
3273    C8274
3274    C8275
3275    C8276
3276    C8277
Name: customer_id, Length: 3277, dtype: object

In [36]:
df_customer[df_customer['customer_id'] == '.']
len(rev_customer)
len(df_customer)
df_customer['customer_sales'].isna().sum()

539

In [86]:
df_customer['customer_sales'] = df_customer['customer_sales'].fillna('.')
df_meta['movie_down_sales'] = df_meta['movie_down_sales'].fillna('.')

In [63]:
df_meta['contract_year']

0            .
1            .
2       2015.0
3       2018.0
4       2014.0
         ...  
5530         .
5531         .
5532         .
5533         .
5534         .
Name: contract_year, Length: 5535, dtype: object

In [62]:
df_meta[df_meta['contract_year'] == '.']


,movie_id,title,release_year,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,legs,...,genre_adult,genre_news,genre_fiction,genre_film-noir,genre_reality-tv,country_1,country_2,country_3,dvd_over_income,contract_price_inf
0,tt4154796,Avengers: Endgame,2019,181.0,PG-13,8.5,19581271.0,69437591.0,89018862.0,2.403631836172037,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.02281207703410988,.
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,136.0,PG-13,6.6,35478573.0,58936309.0,94414882.0,2.673972705063156,...,0.0,0.0,0.0,0.0,0.0,USA,UK,.,0.14717498837185788,.
8,tt0401729,John Carter,2012,132.0,PG-13,6.6,24755272.0,16426306.0,41181578.0,2.4207496321759163,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.33884094728841185,.
10,tt6105098,The Lion King,2019,118.0,PG,6.9,16106589.0,31098360.0,47204949.0,2.8348328276679555,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.029627413326554115,.
11,tt0398286,Tangled,2010,101.0,PG,7.7,179787688.0,65663692.0,245451380.0,4.117983920783237,...,0.0,0.0,0.0,0.0,0.0,USA,.,.,0.8952592111252229,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5530,tt5997666,Jab Harry Met Sejal,2017,144.0,NOT RATED,5.3,.,.,.,1.0,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.
5531,tt5970844,Thugs of Hindostan,2018,164.0,NOT RATED,4.0,.,.,.,1.2231020106530701,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.
5532,tt5956100,Tiger Zinda Hai,2017,145.0,NOT RATED,6.0,.,.,.,3.1116871504128456,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.
5533,tt5882970,Tubelight,2017,170.0,NOT RATED,4.0,.,.,.,1.0,...,0.0,0.0,0.0,0.0,0.0,India,.,.,.,.


In [65]:
# inflation반영 contract_price
df_meta['contract_price_inf'] = '.'
for i in df_meta.index:
    for j in range(len(df_download['down_year'].unique())):
        if df_meta['contract_year'][i] != '.':
            if float(df_meta['contract_year'][i]) == df_download['down_year'].unique()[j]:
                df_meta['contract_price_inf'][i] = float(df_meta['contract_price'][i]) * df_download['inf_year'].unique()[j]
                print(i)


C:\Users\cresc\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


2
3
4
5
6
7
9
12
13
14
15
16
17
19
21
25
26
30
33
35
41
43
45
47
49
52
53
54
66
74
75
76
77
80
85
94
95
100
101
102
103
104
105
116
117
118
119
131
139
140
141
142
156
158
159
160
161
162
164
165
171
172
178
209
230
241
244
246
264
274
287
311
312
314
339
403
434
534
535
538
616
620
623
624
668
669
742
745
774
943
944
970
1063
1069
1346
1582
1583
1584
1594
1953
1974
2096
2737
3906
4991
5071


In [16]:
df_download['down_year'].unique()

array([2014, 2015, 2016, 2017, 2018], dtype=int64)

In [59]:
float(df_meta['contract_year'][12]) == float(df_download['down_year'].unique()[0])

True

In [57]:
df_meta['runtime'][12]

AttributeError: 'str' object has no attribute 'describe'

In [202]:
df_meta[df_meta['inv_exist'] == 1]

,movie_id,title,release_year,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,legs,...,genre_short,genre_game-show,genre_adult,genre_news,genre_fiction,genre_film-noir,genre_reality-tv,dvd_over_income,movie_down_sales,contract_price_inf
2,tt2395427,Avengers: Age of Ultron,2015,141.0,PG-13,7.3,33569045.0,55802159.0,89371204.0,2.399765810946388,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07313423932087944,1042.26,755.774
3,tt4154756,Avengers: Infinity War,2018,156.0,PG-13,8.5,23700665.0,77736363.0,101437028.0,2.6341492753171645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034914738435503155,161,1000
4,tt0449088,Pirates of the Caribbean: At World's End,2007,167.0,PG-13,7.1,314040375.0,7640482.0,321680857.0,2.696878059826299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0149309794271015,1019.24,767.867
5,tt0974015,Justice League,2017,121.0,PG-13,6.4,19449630.0,40475432.0,59925062.0,2.440524623458739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08492387237781913,375.43,745.33
6,tt2379713,Spectre,2015,148.0,PG-13,6.8,12917812.0,24092452.0,37010264.0,2.8418356378041505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064565114413192,472.11,755.774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,tt4846340,Hidden Figures,2016,126.0,PG,7.8,18636040.0,11544851.0,30180891.0,329.0157439684656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10987759034197629,590.648,1034.27
2737,tt6644200,A Quiet Place,2018,95.0,PG-13,7.5,6628735.0,10373895.0,17002630.0,3.7452394513361424,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035254660432006465,159,920
3906,tt5052448,Get Out,2017,104.0,R,7.7,7363633.0,6449455.0,13813088.0,5.274301121788438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04182915918887264,579.142,939.32
4991,tt0121766,Star Wars: Episode III - Revenge of the Sith,2005,139.0,PG-13,7.5,.,.,.,3.506871653257155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.,868.382,967.722


In [191]:
df_meta['contract_year'][2]

2015.0

In [192]:
type(df_download['down_year'].unique()[0])

numpy.int64

In [148]:
df_meta

,movie_id,title,release_year,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,legs,...,genre_documentary,genre_short,genre_game-show,genre_adult,genre_news,genre_fiction,genre_film-noir,genre_reality-tv,dvd_over_income,movie_down_sales
0,tt4154796,Avengers: Endgame,2019,181.0,PG-13,8.5,19581271.0,69437591.0,89018862.0,2.403631836172037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02281207703410988,NaN
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,136.0,PG-13,6.6,35478573.0,58936309.0,94414882.0,2.673972705063156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.14717498837185788,NaN
2,tt2395427,Avengers: Age of Ultron,2015,141.0,PG-13,7.3,33569045.0,55802159.0,89371204.0,2.399765810946388,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07313423932087944,1042.255387
3,tt4154756,Avengers: Infinity War,2018,156.0,PG-13,8.5,23700665.0,77736363.0,101437028.0,2.6341492753171645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034914738435503155,161.000000
4,tt0449088,Pirates of the Caribbean: At World's End,2007,167.0,PG-13,7.1,314040375.0,7640482.0,321680857.0,2.696878059826299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0149309794271015,1019.243399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5530,tt5997666,Jab Harry Met Sejal,2017,144.0,NOT RATED,5.3,.,.,.,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.,NaN
5531,tt5970844,Thugs of Hindostan,2018,164.0,NOT RATED,4.0,.,.,.,1.2231020106530701,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.,NaN
5532,tt5956100,Tiger Zinda Hai,2017,145.0,NOT RATED,6.0,.,.,.,3.1116871504128456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.,NaN
5533,tt5882970,Tubelight,2017,170.0,NOT RATED,4.0,.,.,.,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.,NaN


In [149]:
df_download

,customer_id,item_id,down_year,down_date,weekday,inf_year,down_price
0,C5001,I-1010,2014,02-21-2014,6,1.051872,0.946685
1,C5001,I-1014,2014,04-04-2014,6,1.051872,0.946685
2,C5001,I-1017,2014,01-19-2014,1,1.051872,0.946685
3,C5001,I-1028,2014,01-21-2014,3,1.051872,0.946685
4,C5001,I-1035,2014,03-11-2014,3,1.051872,0.899351
...,...,...,...,...,...,...,...
73158,C8276,I-1101,2018,06-01-2018,6,1.000000,1.000000
73159,C8277,I-1017,2018,06-15-2018,6,1.000000,0.900000
73160,C8277,I-1078,2018,04-19-2018,5,1.000000,0.900000
73161,C8277,I-1099,2018,07-07-2018,7,1.000000,1.000000


In [203]:
df_meta['contract_price_inf']

0             .
1             .
2       755.774
3          1000
4       767.867
         ...   
5530          .
5531          .
5532          .
5533          .
5534          .
Name: contract_price_inf, Length: 5535, dtype: object

In [206]:
df_meta[df_meta['movie_down_sales'] != '.']

,movie_id,title,release_year,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,legs,...,genre_short,genre_game-show,genre_adult,genre_news,genre_fiction,genre_film-noir,genre_reality-tv,dvd_over_income,movie_down_sales,contract_price_inf
2,tt2395427,Avengers: Age of Ultron,2015,141.0,PG-13,7.3,33569045.0,55802159.0,89371204.0,2.399765810946388,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07313423932087944,1042.26,755.774
3,tt4154756,Avengers: Infinity War,2018,156.0,PG-13,8.5,23700665.0,77736363.0,101437028.0,2.6341492753171645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034914738435503155,161,1000
4,tt0449088,Pirates of the Caribbean: At World's End,2007,167.0,PG-13,7.1,314040375.0,7640482.0,321680857.0,2.696878059826299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0149309794271015,1019.24,767.867
5,tt0974015,Justice League,2017,121.0,PG-13,6.4,19449630.0,40475432.0,59925062.0,2.440524623458739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08492387237781913,375.43,745.33
6,tt2379713,Spectre,2015,148.0,PG-13,6.8,12917812.0,24092452.0,37010264.0,2.8418356378041505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064565114413192,472.11,755.774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,tt4846340,Hidden Figures,2016,126.0,PG,7.8,18636040.0,11544851.0,30180891.0,329.0157439684656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10987759034197629,590.648,1034.27
2737,tt6644200,A Quiet Place,2018,95.0,PG-13,7.5,6628735.0,10373895.0,17002630.0,3.7452394513361424,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035254660432006465,159,920
3906,tt5052448,Get Out,2017,104.0,R,7.7,7363633.0,6449455.0,13813088.0,5.274301121788438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04182915918887264,579.142,939.32
4991,tt0121766,Star Wars: Episode III - Revenge of the Sith,2005,139.0,PG-13,7.5,.,.,.,3.506871653257155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,.,868.382,967.722


In [87]:
# 영화별 순수익 (contract_price를 비용으로 볼 때)
df_meta['net_profit'] = '.'
for i in df_meta.index:
    if df_meta['movie_down_sales'][i] != '.':
        df_meta.loc[i,'net_profit'] = df_meta.loc[i,'movie_down_sales'] - df_meta.loc[i,'contract_price_inf']

In [89]:
df_meta[df_meta['net_profit'] != '.']['net_profit'].sum()
print('Net Profit: ${} (2018)'.format(df_meta[df_meta['net_profit'] != '.']['net_profit'].sum()))

Net Profit: $-18424.47258914714 (2018)


In [90]:
# ROI (contract_price를 투자(즉 자산)로 볼 때)
            
total_revenue = df_meta[df_meta['movie_down_sales'] != '.']['movie_down_sales'].sum() # 원래 total_net_profit이 정의에 부합하나 혼동 방지를 위해 revenue로 대체
total_investment = df_meta[df_meta['contract_price_inf'] != '.']['contract_price_inf'].sum()

ROI = float(total_revenue/total_investment)*100
print('ROI: %.2f%%'%ROI)


ROI: 79.45%


In [91]:
total_investment

89669.487685054

In [92]:
total_revenue

71245.0150959068